In [1]:
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

from keras.preprocessing.image import ImageDataGenerator

# Importing the Keras libraries and packages
from keras.models import Sequential

# Required for Step 1 - Convolution
from keras.layers import Conv2D

# Required for Step 2 - Pooling 
from keras.layers import MaxPooling2D

# Required for Step 3 - Flattening
from keras.layers import Flatten

# Required for Step 4 - Full connection ANN - You have already used this
from keras.layers import Dense

seed=47

2.2.0


In [2]:
import mlflow
from azureml.core import Workspace

ws = Workspace.from_config(_file_name='config.json')

In [3]:
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

experiment_name = 'Autonomyous-Vehicle-Stop-Sign-Classifier'
mlflow.set_experiment(experiment_name)

In [4]:
from mlflow.keras import log_model

In [5]:
datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.2)

In [16]:
training_set = datagen.flow_from_directory('./GTSRB_Train/Final_Training/Images',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 subset = 'training',
                                                 seed=seed)

Found 28374 images belonging to 37 classes.


In [17]:
test_set = datagen.flow_from_directory('./GTSRB_Train/Final_Training/Images',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            # class_mode = 'binary',
                                            subset = 'validation',
                                            seed=seed)

Found 7092 images belonging to 37 classes.


In [18]:
from keras.layers import Dropout

# Initialize the CNN
neural = Sequential()

#Convolution and Pooling Layers
neural.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
neural.add(MaxPooling2D(pool_size = (2, 2)))
neural.add(Conv2D(32, (3, 3), activation = 'relu'))
neural.add(MaxPooling2D(pool_size = (2, 2)))
neural.add(Flatten())

#Hidden Layers
neural.add(Dense(units = 128, activation = 'relu'))
neural.add(Dropout(0.5))
neural.add(Dense(units = 128, activation = 'relu'))
neural.add(Dropout(0.5))
neural.add(Dense(units = 128, activation = 'relu'))
neural.add(Dropout(0.5))
neural.add(Dense(units = 128, activation = 'relu'))

#Output Layer
neural.add(Dense(units = 37, activation = 'softmax'))

In [19]:
## Add Early Stopping
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(patience=3, monitor='loss', min_delta=0.001)

## Callbacks can be a list
clf_callbacks = [earlystop]

In [20]:
from mlflow.keras import log_model

def neuralRunner(run_name, compile_arguments, fit_arguments):

  with mlflow.start_run(run_name=run_name) as run:
    model = neural
    model.compile(**compile_arguments)
    
    history = model.fit_generator(**fit_arguments)
    
    for param_key, param_value in {**compile_arguments, **fit_arguments}.items():
      if param_key not in ["x", "y"]:
        mlflow.log_param(param_key, param_value)
    
    for key, values in history.history.items():
      for i, v in enumerate(values):
        mlflow.log_metric(key, v, step=i)

    for i, layer in enumerate(model.layers):
      mlflow.log_param(f"hidden_layer_{i}_units", layer.output_shape)
      
    log_model(model, "stop_classifier")
        
    return run

In [25]:
# We have to use categorical cross-entropy loss
compile_arguments = {
  "optimizer": "adam", 
  "loss": "categorical_crossentropy",
  "metrics": ["accuracy"],
}

# Specify arguments to fit the Keras classifier model - it would look like below
# classifier.fit_generator(training_set, steps_per_epoch = 8000, epochs = 8, 
# validation_data = test_set, validation_steps = 2000, callbacks=clf_callbacks)
fit_arguments = {
  "generator": training_set, 
  "steps_per_epoch": 221,
  "epochs": 10,
  "validation_data": test_set,
  "validation_steps": 2000, 
  "callbacks": clf_callbacks,
  "verbose": 2,
  "use_multiprocessing":False
}

In [26]:
## Import Python datetime to create a runname that includes the datetime when it was started
from datetime import datetime

now = datetime.now() # current date and time
datetime_str = now.strftime("%m%d%Y_%H%M%S")
print(datetime_str)                            

07162020_131344


In [27]:
ml_run_name = "AV_stop_classifier" + datetime_str
print("ML run name: ", ml_run_name)

ML run name:  AV_stop_classifier07162020_131344


In [28]:
run_name = ml_run_name
run = neuralRunner(run_name, compile_kwargs, fit_kwargs)

Epoch 1/10


WARNING - Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 2000 batches). You may need to use the repeat() function when building your dataset.


221/221 - 22s - loss: 2.0268 - accuracy: 0.3186 - val_loss: 1.8624 - val_accuracy: 0.3503
Epoch 2/10


WARNING - Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 2000 batches). You may need to use the repeat() function when building your dataset.


221/221 - 22s - loss: 1.9616 - accuracy: 0.3281 - val_loss: 1.8229 - val_accuracy: 0.3589
Epoch 3/10


...

WARNING - Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 2000 batches). You may need to use the repeat() function when building your dataset.


221/221 - 22s - loss: 1.7163 - accuracy: 0.3978 - val_loss: 1.6208 - val_accuracy: 0.4199
Epoch 10/10


WARNING - Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 2000 batches). You may need to use the repeat() function when building your dataset.


221/221 - 22s - loss: 1.7159 - accuracy: 0.4070 - val_loss: 1.6341 - val_accuracy: 0.4213


2020/07/16 13:18:08 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under azureml://experiments/Autonomyous-Vehicle-Stop-Sign-Classifier/runs/45f7fd75-5b08-434d-9549-9569371070e3/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the  tracking store. If logging to a mlflow server via REST, consider  upgrading the server version to MLflow 1.7.0 or above.
